# Clustering Merch Data Shirts
1. read in data file and create large list of indexed data
2. remove texts that pop up in all documents (i.e. stopwords / phrases)
3. clean the documents
4. stem and tokenize the words in each document
5. run all documents through tf-idf vectorizer

To Do:
- read in datafiles better (use different delimiter?)
- clean up input documents better
- possibly create my own tfidf vectorizor that works as a sum rather than a median


## Initialize steps

First we import the necessary libraries.

In [47]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

Import the stemmer and then tokenize and stem the text.

In [48]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if not re.search('[^a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if not re.search('[^a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens
print(tokenize_and_stem('artwork'))
# totalvocab_stemmed = []
# totalvocab_tokenized = []
# for i in documents:
#     allwords_stemmed = tokenize_and_stem(i)
#     totalvocab_stemmed.extend(allwords_stemmed)
    
#     allwords_tokenized = tokenize_only(i)
#     totalvocab_tokenized.extend(allwords_tokenized)
    
# vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

['artwork']


In [49]:
print(tokenize_and_stem('retro'))

['retro']


Now we import the shirt data file and create documents from it. Each document is the title of the shirt followed by it's description.

In [50]:
def clean_string(document):
    document = document.lower()
    default_1 = "Lightweight, Classic fit, Double-needle sleeve and bottom hem".lower()
    default_2 = "Solid colors: 100% Cotton; Heather Grey: 90% Cotton, 10% Polyester; All Other Heathers: 50% Cotton, 50% Polyester Imported Machine wash cold with like colors, dry low heat ".lower()
    document = document.replace(default_1, '').replace(default_2,'')
    
    document.replace("tee shirt","tshirt")
    document.replace("t-shirt", 'tshirt')
    document.replace(' t shirt', 'tshirt')
    document.replace("-", " ")
    
    document.replace("shirt", "")
    document.replace("tshirt", "")
    
    document.replace("officially licensed","")
    
    if re.compile("officially licensed [\w\s]+ (apparel|shirt)").search(document):
        document = re.sub(r'officially licensed [\w\s]+ (apparel|shirt)', '', document)
    
    if re.compile("official [\w\s]+merchandise").search(document):
        document = re.sub(r'official [\w\s]+merchandise', '', document)

    if re.compile("graphic [\w\s\-]+shirt").search(document):
        document = re.sub(r'graphic [\w\s\-]+shirt', '', document)
    
    
    document = BeautifulSoup(document, 'html.parser').getText()
    
    return document

document_data_dict = {}
def read_shirt_data_file(path):
    document_data = []
    with open(path, 'r') as data_file:
        for line in data_file:
            data = {}
            if len(line.replace("\"","").split('|')) == 11:
                for item in line.replace("\"","").split('|'):
                    data[item.split(':',1)[0]] = item.split(':',1)[1]
                data['document'] = clean_string(data['title']) # + ". " + data['description'])
    #             line = line.split('|',10)
    #             new_line = []
    #             for item in line:
    #                 print(item)
    #                 new_line.append(item.split(':',1)[1])
    #             line = [item.split(':',1)[1] for item in line]
    #             line.append(clean_string(line[2] + ". " + line[10]))
                if data['asin'] not in document_data_dict:
                    document_data_dict[data['asin']] = data
                    document_data.append(data)
                
    # [salesRank (0), asin(1), title(2), imageUrl(3), trademarked(4), isMerch(5), date(6), unix(7), errorMessage(8), link(9), description(10), document(11)]
    return document_data

document_data = read_shirt_data_file("shirts_newest_nt") # read_shirt_data_file("shirts_featured_nt")
print(document_data[0])
print(len(document_data))

{'salesRank': 'NA', 'asin': 'B07X97D31R', 'title': 'Support Cancer Shirt Prostate Cancer Awareness Tshirt', 'imgUrl': 'https://m.media-amazon.com/images/I/A13usaonutL._CLa%7C2140%2C2000%7C918G0EUHpmL.png%7C0%2C0%2C2140%2C2000%2B0.0%2C0.0%2C2140.0%2C2000.0._UX342_.png', 'trademarked': 'False', 'isMerch': 'True', 'date': 'August 30, 2019', 'unix': '1567123200', 'errorMessage': '[]', 'link': 'https://www.amazon.com/dp/B07X97D31R', 'description': 'Solid colors: 100% Cotton; Heather Grey: 90% Cotton, 10% Polyester; All Other Heathers: 50% Cotton, 50% Polyester Imported Machine wash cold with like colors, dry low heat Are you a Fighter who has or is fighting Prostate Cancer? This Motivational T-shirt is perfect for you. Great shirt for your Light Blue ribbon events, Hospital Visits or To support a loved one with Cancer. This Prostate Cancer Tshirt is a great Birthday or Christmas Gift For Surviors. Show your love with a Light Blue Ribbon. Lightweight, Classic fit, Double-needle sleeve and bo

Clean the documents to make them unicode.

In [51]:
# def clean_documents(documents):
#     documents_clean = []
#     for text in documents:
#         text = BeautifulSoup(text, 'html.parser').getText()
#         documents_clean.append(text)

#     return documents_clean

# documents = clean_documents(documents)

In [52]:
# print(documents)

## Option 1: 
cosine similarity of stems and ngrams (Tf-idf and document similarity)

### Create tfidf vector from documents

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

with open('stop_words.txt','r') as f:
    extra_stop_words = f.read().split('\n')

print(extra_stop_words)

my_stop_words = text.ENGLISH_STOP_WORDS.union(extra_stop_words)
#print(my_stop_words)

tfidf_vectorizer = TfidfVectorizer(max_df=0.02, max_features=10000000,
                                 min_df=0, stop_words=my_stop_words,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform([document['document'] for document in document_data])
#print(tfidf_matrix[0])
print(tfidf_matrix.shape)

terms = tfidf_vectorizer.get_feature_names()
print(len(terms))
#print(stemmed_documents[99])

print(terms[247])



['logo', 'shirt', 'tshirt', 'vintag', 'retro', 'movie', 'officially', 'licensed', 'official', 'offici', 'licens', 'graphic', 'artwork', 'gift', 'men', 'women', '']


/usr/local/anaconda3/envs/niche_finder/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'licen', 'mani', 'meanwhil', 'moreov', 'movi', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


CPU times: user 4.82 s, sys: 38.1 ms, total: 4.86 s
Wall time: 4.84 s
(16010, 74640)
74640
adjust


In [54]:
# Testing cosine similarity comparisons

from sklearn.metrics.pairwise import cosine_similarity
#print(cosine_similarity(tfidf_matrix))
# print(tfidf_matrix[0])
# print(tfidf_matrix[1])
# print(tfidf_matrix[2])

start = 0
length = 1
dist = cosine_similarity(tfidf_matrix[start:start+length],tfidf_matrix)
print(dist[0])
for doc_vector_index, doc_vector in enumerate(dist):
    print("checking {} for similarities...".format(document_data[start + doc_vector_index]['asin']))
    print("{}".format(document_data[start + doc_vector_index]['document']))
    for similarity_index, doc_similarity in enumerate(doc_vector):
        if doc_similarity > .2:
            print("{} {}".format(doc_similarity, document_data[similarity_index]['document']))
        

# print(tfidf_matrix.shape)
# print(dist)

[1. 0. 0. ... 0. 0. 0.]
checking B07X97D31R for similarities...
support cancer shirt prostate cancer awareness tshirt
1.0 support cancer shirt prostate cancer awareness tshirt
0.22020866476987808 breast cancer awareness support tee
0.2165820341351916 strength against cancer vintage childhood cancer awareness t-shirt
0.21020493217342937 prostate cancer sucks dinosaur trex blue ribbon awareness
0.2140131736540844 breast cancer awareness shirt breast cancer shirts for women t-shirt
0.3377413328599886 i wear light blue for my dad prostate cancer awareness shirt
0.35034916405125105 his fight is my fight i prostate cancer awareness fight gift t-shirt
0.21563231640052763 fuck cancer tshirt awareness for cancer survivor gifts t-shirt
0.3755102199922954 i wear blue for my uncle prostate cancer awareness shirt
0.42907667964294915 wolf still here still fighting prostate cancer awareness t-shirt


### Compute cosine similarity between all doc vectors, and create niches

In [55]:
from sklearn.metrics.pairwise import cosine_similarity
import pprint
pp = pprint.PrettyPrinter(indent=4)

niches = {}

dist = cosine_similarity(tfidf_matrix)

for doc_vector_index, doc_vector in enumerate(dist):
#     if doc_vector_index > 100:
#         break

    niches[doc_vector_index] = {}
    niches[doc_vector_index]['similar_docs'] = set()
    niches[doc_vector_index]['percent_sales_ranks'] = 0.0
    niches[doc_vector_index]['average_sales_rank'] = 0
    niches[doc_vector_index]['hot'] = False
    niches[doc_vector_index]['consumed'] = False
    for similar_doc_index, similar_doc in enumerate(doc_vector):
        if similar_doc > .2:
            niches[doc_vector_index]['similar_docs'].add(similar_doc_index)
            # if doc_vector_index != similarity_index and similarity_index in niches:
                
    # if float(len(niches[doc_vector_index].intersection(niches[similarity_index])))/float(min(len(niches[similarity_index]),len(niches[doc_vector_index]))) > 0.75

    for similar_doc in niches[doc_vector_index]['similar_docs']:
        if doc_vector_index != similar_doc and similar_doc in niches and not niches[similar_doc]['consumed']:

            intersecting_docs = niches[doc_vector_index]['similar_docs'].intersection(niches[similar_doc]['similar_docs'])
            smaller_niche = min(len(niches[similar_doc]['similar_docs']),len(niches[doc_vector_index]['similar_docs']))                                                                          

            if (float(len(intersecting_docs))/float(smaller_niche)) > 0.75:
#                 print("these two share more than 75% of items")
#                 print(float(len(niches[doc_vector_index]['similar_docs'])))
#                 print(doc_vector_index)
#                 print(niches[doc_vector_index]['similar_docs'])
#                 print(document_data[doc_vector_index]['document'])
#                 print(float(len(niches[similar_doc])))
#                 print(similar_doc)
#                 print(niches[similar_doc]['similar_docs'])
#                 print(document_data[similar_doc]['document'])
#                 for item in niches[similar_doc]:
#                     print(item)
#                     print(document_data[item]['document'])
                if len(niches[doc_vector_index]['similar_docs']) < len(niches[similar_doc]['similar_docs']):
                    print("consuming {}".format(doc_vector_index))
                    niches[doc_vector_index]['consumed'] = True
                else:
                    print("consuming {}".format(similar_doc))
                    niches[similar_doc]['consumed'] = True
                
                    

# pp.pprint(niches)

    
            
            


consuming 6
consuming 26
consuming 35
consuming 41
consuming 23
consuming 38
consuming 57
consuming 48
consuming 63
consuming 60
consuming 69
consuming 58
consuming 70
consuming 80
consuming 85
consuming 3
consuming 55
consuming 51
consuming 62
consuming 95
consuming 99
consuming 107
consuming 110
consuming 82
consuming 66
consuming 124
consuming 118
consuming 119
consuming 130
consuming 121
consuming 125
consuming 89
consuming 135
consuming 91
consuming 54
consuming 157
consuming 134
consuming 158
consuming 162
consuming 15
consuming 167
consuming 147
consuming 169
consuming 155
consuming 166
consuming 174
consuming 176
consuming 177
consuming 178
consuming 168
consuming 182
consuming 190
consuming 194
consuming 161
consuming 209
consuming 212
consuming 214
consuming 215
consuming 217
consuming 198
consuming 228
consuming 233
consuming 237
consuming 165
consuming 252
consuming 164
consuming 257
consuming 259
consuming 239
consuming 264
consuming 242
consuming 271
consuming 262
consumi

consuming 1254
consuming 910
consuming 1419
consuming 1423
consuming 1206
consuming 1427
consuming 1428
consuming 1430
consuming 1431
consuming 1158
consuming 923
consuming 884
consuming 1443
consuming 1445
consuming 1447
consuming 369
consuming 1376
consuming 1453
consuming 78
consuming 1455
consuming 1456
consuming 1197
consuming 1462
consuming 625
consuming 1426
consuming 1465
consuming 874
consuming 1468
consuming 467
consuming 1469
consuming 365
consuming 1475
consuming 1476
consuming 1479
consuming 1480
consuming 1337
consuming 1401
consuming 1485
consuming 1486
consuming 1492
consuming 1494
consuming 1446
consuming 1139
consuming 1501
consuming 797
consuming 1208
consuming 1507
consuming 1508
consuming 1509
consuming 1512
consuming 1513
consuming 1515
consuming 1141
consuming 1517
consuming 1518
consuming 1523
consuming 1523
consuming 204
consuming 1525
consuming 1527
consuming 1383
consuming 1529
consuming 1538
consuming 1170
consuming 1488
consuming 1544
consuming 1545
consumi

consuming 2400
consuming 2409
consuming 391
consuming 2088
consuming 1130
consuming 2318
consuming 2416
consuming 2417
consuming 2417
consuming 2417
consuming 2418
consuming 2421
consuming 2423
consuming 2423
consuming 2306
consuming 129
consuming 2432
consuming 2433
consuming 2434
consuming 2435
consuming 2438
consuming 2350
consuming 1305
consuming 2371
consuming 2445
consuming 2446
consuming 1328
consuming 2448
consuming 265
consuming 2450
consuming 1156
consuming 1696
consuming 2457
consuming 2460
consuming 2461
consuming 2463
consuming 1651
consuming 2465
consuming 2466
consuming 2468
consuming 2468
consuming 2472
consuming 2475
consuming 1415
consuming 2477
consuming 2478
consuming 2424
consuming 2325
consuming 2483
consuming 2331
consuming 1939
consuming 532
consuming 488
consuming 2304
consuming 2491
consuming 2492
consuming 2395
consuming 2498
consuming 2481
consuming 2420
consuming 2506
consuming 2507
consuming 2509
consuming 2510
consuming 2510
consuming 2516
consuming 2521


consuming 3300
consuming 2176
consuming 3306
consuming 1296
consuming 3308
consuming 3309
consuming 2612
consuming 3312
consuming 3130
consuming 1759
consuming 3320
consuming 3321
consuming 3322
consuming 3282
consuming 3324
consuming 3326
consuming 645
consuming 3207
consuming 3329
consuming 3330
consuming 3331
consuming 3332
consuming 755
consuming 3333
consuming 3334
consuming 2112
consuming 3336
consuming 3338
consuming 3338
consuming 3338
consuming 3249
consuming 3342
consuming 1129
consuming 3345
consuming 3347
consuming 3348
consuming 2459
consuming 355
consuming 2484
consuming 3358
consuming 3358
consuming 2356
consuming 3361
consuming 2824
consuming 3135
consuming 3363
consuming 3364
consuming 3365
consuming 820
consuming 1297
consuming 3370
consuming 3370
consuming 3261
consuming 3241
consuming 3374
consuming 3377
consuming 3379
consuming 3380
consuming 3386
consuming 457
consuming 3389
consuming 3392
consuming 1791
consuming 3394
consuming 3395
consuming 3395
consuming 3396


consuming 2627
consuming 4126
consuming 4129
consuming 4130
consuming 4131
consuming 3931
consuming 4135
consuming 4137
consuming 4137
consuming 3908
consuming 4139
consuming 4139
consuming 3166
consuming 4141
consuming 4142
consuming 4144
consuming 4145
consuming 4145
consuming 4145
consuming 4146
consuming 4149
consuming 4069
consuming 3154
consuming 1720
consuming 4158
consuming 4161
consuming 4162
consuming 4164
consuming 4165
consuming 1690
consuming 4169
consuming 4173
consuming 4175
consuming 3879
consuming 4183
consuming 4185
consuming 3430
consuming 4052
consuming 4189
consuming 4184
consuming 4192
consuming 4193
consuming 1972
consuming 4035
consuming 4196
consuming 3606
consuming 4202
consuming 4204
consuming 4204
consuming 2296
consuming 4206
consuming 4207
consuming 4207
consuming 4208
consuming 4209
consuming 4209
consuming 4210
consuming 2487
consuming 4072
consuming 4213
consuming 453
consuming 2473
consuming 4217
consuming 2698
consuming 4219
consuming 4224
consuming 4

...

consuming 12924
consuming 13079
consuming 13255
consuming 13256
consuming 13257
consuming 13145
consuming 13261
consuming 13261
consuming 13261
consuming 13261
consuming 13263
consuming 13265
consuming 13254
consuming 13267
consuming 13267
consuming 13010
consuming 13269
consuming 13269
consuming 13271
consuming 13273
consuming 13276
consuming 13279
consuming 13222
consuming 13282
consuming 13282
consuming 13282
consuming 13282
consuming 13283
consuming 13284
consuming 13284
consuming 13284
consuming 13285
consuming 13285
consuming 2984
consuming 11319
consuming 13289
consuming 13292
consuming 13294
consuming 13295
consuming 13027
consuming 13299
consuming 13300
consuming 13300
consuming 13301
consuming 13302
consuming 7376
consuming 12191
consuming 13304
consuming 13305
consuming 13305
consuming 13247
consuming 13307
consuming 12025
consuming 13308
consuming 12192
consuming 3807
consuming 4085
consuming 13311
consuming 13194
consuming 13313
consuming 13314
consuming 13316
consuming 13

consuming 10582
consuming 13868
consuming 13870
consuming 13870
consuming 13870
consuming 13871
consuming 13871
consuming 13871
consuming 13873
consuming 13875
consuming 13875
consuming 13875
consuming 6826
consuming 12473
consuming 13878
consuming 13878
consuming 13878
consuming 13879
consuming 9940
consuming 13880
consuming 13886
consuming 13887
consuming 13887
consuming 3633
consuming 13890
consuming 13890
consuming 13890
consuming 13893
consuming 13893
consuming 13894
consuming 13895
consuming 13896
consuming 13897
consuming 13897
consuming 13898
consuming 13899
consuming 13531
consuming 8355
consuming 13904
consuming 13905
consuming 13907
consuming 13908
consuming 13909
consuming 10782
consuming 9897
consuming 12986
consuming 13916
consuming 13381
consuming 8512
consuming 13921
consuming 13922
consuming 13741
consuming 4621
consuming 5182
consuming 5665
consuming 5240
consuming 13929
consuming 9602
consuming 13861
consuming 4200
consuming 13576
consuming 11268
consuming 13935
cons

consuming 14462
consuming 14464
consuming 14465
consuming 14465
consuming 14465
consuming 14465
consuming 14466
consuming 14467
consuming 6443
consuming 14469
consuming 14470
consuming 14470
consuming 14471
consuming 12670
consuming 14473
consuming 14474
consuming 14477
consuming 14478
consuming 14478
consuming 14479
consuming 14480
consuming 14481
consuming 14133
consuming 14484
consuming 14485
consuming 9030
consuming 13484
consuming 13627
consuming 14488
consuming 14491
consuming 14492
consuming 14492
consuming 14492
consuming 14494
consuming 14495
consuming 4178
consuming 14498
consuming 14499
consuming 14502
consuming 14504
consuming 13016
consuming 14506
consuming 14507
consuming 13615
consuming 13620
consuming 14509
consuming 13435
consuming 14513
consuming 14514
consuming 14515
consuming 14516
consuming 11944
consuming 14516
consuming 14516
consuming 14518
consuming 14519
consuming 14520
consuming 14521
consuming 14312
consuming 14524
consuming 14524
consuming 5821
consuming 14

consuming 5330
consuming 14965
consuming 15020
consuming 15021
consuming 15021
consuming 15021
consuming 15021
consuming 15022
consuming 15023
consuming 15024
consuming 15024
consuming 14942
consuming 15026
consuming 15027
consuming 15027
consuming 14880
consuming 15029
consuming 5974
consuming 11249
consuming 4914
consuming 14862
consuming 15035
consuming 14995
consuming 10209
consuming 15041
consuming 15041
consuming 15042
consuming 13632
consuming 13491
consuming 13589
consuming 12601
consuming 5078
consuming 15051
consuming 15052
consuming 9973
consuming 970
consuming 14883
consuming 15057
consuming 15057
consuming 15057
consuming 14611
consuming 15061
consuming 15062
consuming 15062
consuming 15064
consuming 15066
consuming 15068
consuming 9432
consuming 12229
consuming 4954
consuming 15079
consuming 15081
consuming 15081
consuming 15082
consuming 15082
consuming 15083
consuming 15083
consuming 15084
consuming 7955
consuming 14700
consuming 15086
consuming 15045
consuming 15089
co

consuming 15610
consuming 15610
consuming 15612
consuming 15615
consuming 15616
consuming 15617
consuming 15617
consuming 15437
consuming 15619
consuming 15618
consuming 15322
consuming 15376
consuming 15407
consuming 15625
consuming 15050
consuming 15627
consuming 15628
consuming 14609
consuming 15630
consuming 15632
consuming 15633
consuming 15634
consuming 15512
consuming 15637
consuming 15638
consuming 15638
consuming 15033
consuming 710
consuming 15640
consuming 15641
consuming 15641
consuming 14763
consuming 15221
consuming 15644
consuming 15511
consuming 15647
consuming 15648
consuming 15650
consuming 15654
consuming 12019
consuming 15656
consuming 15656
consuming 15658
consuming 15659
consuming 15660
consuming 15352
consuming 15585
consuming 664
consuming 15056
consuming 9214
consuming 15657
consuming 14783
consuming 14724
consuming 15670
consuming 4645
consuming 15673
consuming 15673
consuming 15674
consuming 15492
consuming 15674
consuming 15675
consuming 15285
consuming 1348

### Analyze and export niches to textfile

In [61]:
def analyze_niche(niche):
    total_sales_rank = 0
    num_sales_ranks = 0
    for document in niche['similar_docs']:
        if document_data[document]['salesRank'] != "NA":
            num_sales_ranks += 1
            total_sales_rank += int(document_data[document]['salesRank'])
            if 'best_sales_rank' in niche and int(document_data[document]['salesRank']) < int(niche['best_sales_rank']):
                niche['best_sales_rank'] = document_data[document]['salesRank']
            elif 'best_sales_rank' not in niche:
                niche['best_sales_rank'] = document_data[document]['salesRank']
                
    niche['percent_sales_ranks'] = float(num_sales_ranks) / float(len(niche['similar_docs']))
    if num_sales_ranks > 0:
        niche['average_sales_rank'] = float(total_sales_rank) / float(num_sales_ranks)

def hot_niche(niche):
    if len(niche['similar_docs']) > 5:

        if niche['percent_sales_ranks'] > 0.8 and niche['average_sales_rank'] < 1500000:
            return True
    
        if niche['percent_sales_ranks'] > 0.5 and niche['average_sales_rank'] < 900000:
            return True
    
    return False

num_hot_niches = 0
for niche in niches:
    if len(niches[niche]['similar_docs']) > 0 and niches[niche]['consumed'] is False:
        analyze_niche(niches[niche])
        if hot_niche(niches[niche]):
            num_hot_niches += 1
            niches[niche]['hot'] = True

print(num_hot_niches)
with open("niches.txt", 'w') as f:
    for niche in niches:
        if niches[niche]['hot'] and not niches[niche]['consumed']:
            f.write("number of documents in cluster: " + str(len(niches[niche]['similar_docs'])) + '\n')
            f.write("percent sales ranks: " + str(round(niches[niche]['percent_sales_ranks'],2)) + "\n")
            f.write("average sales rank: " + str(round(niches[niche]['average_sales_rank'],2)) + "\n")
            f.write("best sales rank: " + str(niches[niche]['best_sales_rank']) + "\n")
            for document in niches[niche]['similar_docs']:
                f.write(document_data[document]['asin'] + ', ')
            f.write("\n")
            for document in niches[niche]['similar_docs']:
                f.write(document_data[document]['document'] + "\n")
            f.write("\n")

61


## Option 2: 
soft cosine similarity using fasttext similarity matrix of single tokens (not good for mba because the exact text needs to be the same)

In [ ]:
import gensim
# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess # all this does is split a string into an array and clean out punctuation
print(gensim.__version__)
#> '3.6.0'

# Download the FastText model
og_fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')
print("done")

In [ ]:
print(documents[0])
stemmed_documents = [tokenize_and_stem(document) for document in documents ]
print(stemmed_documents[0])

# simple_preprocess

# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary(stemmed_documents) # should be using just documents
print(dictionary)
# Prepare the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)
print(similarity_matrix)
# Convert the sentences into bag-of-words vectors.
sent_1 = dictionary.doc2bow(simple_preprocess(documents[0]))
print(sent_1)

In [ ]:
print(dictionary.token2id)

In [ ]:
print(similarity_matrix[7])

In [ ]:
sent_1 = dictionary.doc2bow(simple_preprocess(documents[0]))
print(sent_1)
sent_2 = dictionary.doc2bow(simple_preprocess(documents[16]))
print(documents[0])
print(documents[18])
print(softcossim(sent_1, sent_2, similarity_matrix))

## K-means clustering

K-means isn't necessarily the best. You must initialize with a pre-determined number of clusters

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

## Hierarchial document clustering

In [ ]:
from scipy.cluster.hierarchy import ward, dendrogram
import matplotlib.pyplot as plt


linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=asins);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
plt.savefig('ward_clusters.png', dpi=200) #save figure as ward_clusters